<a href="https://colab.research.google.com/github/jahelsantiago/tensorflow_tutorial/blob/master/RNN_predicted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np 

In [55]:
data="In the town of Athy one Jeremy Lanigan \n Battered away til he hadnt a pound. \nHis father died and made him a man again \n Left him a farm and ten acres of ground. \nHe gave a grand party for friends and relations \nWho didnt forget him when come to the wall, \nAnd if youll but listen Ill make your eyes glisten \nOf the rows and the ructions of Lanigans Ball. \nMyself to be sure got free invitation, \nFor all the nice girls and boys I might ask, \nAnd just in a minute both friends and relations \nWere dancing round merry as bees round a cask. \nJudy ODaly, that nice little milliner, \nShe tipped me a wink for to give her a call, \nAnd I soon arrived with Peggy McGilligan \nJust in time for Lanigans Ball. \nThere were lashings of punch and wine for the ladies, \nPotatoes and cakes; there was bacon and tea, \nThere were the Nolans, Dolans, OGradys \nCourting the girls and dancing away. \nSongs they went round as plenty as water, \nThe harp that once sounded in Taras old hall,\nSweet Nelly Gray and The Rat Catchers Daughter,\nAll singing together at Lanigans Ball. \nThey were doing all kinds of nonsensical polkas \nAll round the room in a whirligig. \nJulia and I, we banished their nonsense \nAnd tipped them the twist of a reel and a jig. \nAch mavrone, how the girls got all mad at me \nDanced til youd think the ceiling would fall. \nFor I spent three weeks at Brooks Academy \nLearning new steps for Lanigans Ball. \nThree long weeks I spent up in Dublin, \nThree long weeks to learn nothing at all,\n Three long weeks I spent up in Dublin, \nLearning new steps for Lanigans Ball. \nShe stepped out and I stepped in again, \nI stepped out and she stepped in again, \nShe stepped out and I stepped in again, \nLearning new steps for Lanigans Ball. \nBoys were all merry and the girls they were hearty \nAnd danced all around in couples and groups, \nTil an accident happened, young Terrance McCarthy \nPut his right leg through miss Finnertys hoops. \nPoor creature fainted and cried Meelia murther, \nCalled for her brothers and gathered them all. \nCarmody swore that hed go no further \nTil he had satisfaction at Lanigans Ball. \nIn the midst of the row miss Kerrigan fainted, \nHer cheeks at the same time as red as a rose. \nSome of the lads declared she was painted, \nShe took a small drop too much, I suppose. \nHer sweetheart, Ned Morgan, so powerful and able, \nWhen he saw his fair colleen stretched out by the wall, \nTore the left leg from under the table \nAnd smashed all the Chaneys at Lanigans Ball. \nBoys, oh boys, twas then there were runctions. \nMyself got a lick from big Phelim McHugh. \nI soon replied to his introduction \nAnd kicked up a terrible hullabaloo. \nOld Casey, the piper, was near being strangled. \nThey squeezed up his pipes, bellows, chanters and all. \nThe girls, in their ribbons, they got all entangled \nAnd that put an end to Lanigans Ball."

tokenizer = Tokenizer(oov_token = "<OOV>")
sentences = data.lower().split("\n")

tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)

max_lenght = max([len(x) for x in sequences]) #longitud maxima de la sequencia que sera usada en el padded de la misma
word_index = tokenizer.word_index #diccionario donde se almacenan las palabras 
vocab_size = len(word_index)+1 #se suma uno para incluir el <OOV>

#añadimos las pirmeras dos palabras, luego las siguentes 3, luego las siguientes 4 y asi sucesivamente para que sea mas facil prosesarlo como train y test
train = []
for line in sequences:
  temp = []
  for i in range(2,len(line)):
    temp = line[:i]    
    train.append(temp)

train = pad_sequences(train, maxlen= max_lenght)

In [41]:
xs = train[:,:-1] #de los que tomamos en el paso anterior seleccionaos todos menos el ultimo
train_labels = train[:,-1] #el ultimos sera nuetro label de entrenamiento

ys = tf.keras.utils.to_categorical(train_labels) #funcion de keras para convertir los label en one hot, donde tendra la dimension del vocab_size

In [42]:
model = Sequential()
model.add(Embedding(vocab_size, 64, input_length= max_lenght))
model.add(Bidirectional(LSTM(20)))
model.add(Dense(vocab_size, activation = "softmax"))

model.compile("adam", "categorical_crossentropy", metrics = ["accuracy"])

history = model.fit(xs, ys, epochs=500)

Epoch 1/500
32/32 [==============================] - 1s 35ms/step - loss: 6.1761 - accuracy: 0.0271
Epoch 2/500
32/32 [==============================] - 1s 34ms/step - loss: 5.8328 - accuracy: 0.0603
Epoch 3/500
32/32 [==============================] - 1s 35ms/step - loss: 5.4277 - accuracy: 0.0603
Epoch 4/500
32/32 [==============================] - 1s 35ms/step - loss: 5.3619 - accuracy: 0.0603
Epoch 5/500
32/32 [==============================] - 1s 35ms/step - loss: 5.3424 - accuracy: 0.0603
Epoch 6/500
32/32 [==============================] - 1s 36ms/step - loss: 5.3228 - accuracy: 0.0603
Epoch 7/500
32/32 [==============================] - 1s 36ms/step - loss: 5.2946 - accuracy: 0.0603
Epoch 8/500
32/32 [==============================] - 1s 35ms/step - loss: 5.2593 - accuracy: 0.0603
Epoch 9/500
32/32 [==============================] - 1s 36ms/step - loss: 5.2186 - accuracy: 0.0603
Epoch 10/500
32/32 [==============================] - 1s 36ms/step - loss: 5.1806 - accuracy: 0.0673

In [43]:
seed_text = "Mi estimado juanca el ser y tiempo es tan sencillo como"
next_words = 100
word_index_inverse = {i:word for word, i in word_index.items()}  


for _ in range(next_words):
  token_list = tokenizer.texts_to_sequences([seed_text])[0]
  token_list = pad_sequences([token_list], maxlen=max_lenght)
  predicted = int(model.predict_classes(token_list, verbose=0)[0])
  output_word = str(word_index_inverse[predicted])
  seed_text += " " + output_word



In [44]:
print(seed_text)

Mi estimado juanca el ser y tiempo es tan sencillo como el más universal ello no puede signi‐ ficar que sea el más claro y que no esté necesitado de una discusión de que no esté necesitado de una discusión de “ser” si hace más “obvio” y que no sólo falta al terreno de esta expresión resulta comprensible “sin “el “sin antigua— antigua— “el es comprensible uni‐ legitimación insiste los co‐ co‐ ella procedimiento es es es es es es es es es es es es es es es es es es es es es es es es es es es es es es es es es es es es es


{1: '<OOV>',
 2: 'and',
 3: 'the',
 4: 'a',
 5: 'in',
 6: 'all',
 7: 'i',
 8: 'for',
 9: 'of',
 10: 'lanigans',
 11: 'ball',
 12: 'were',
 13: 'at',
 14: 'to',
 15: 'she',
 16: 'stepped',
 17: 'his',
 18: 'girls',
 19: 'as',
 20: 'they',
 21: 'til',
 22: 'he',
 23: 'again',
 24: 'got',
 25: 'boys',
 26: 'round',
 27: 'that',
 28: 'her',
 29: 'there',
 30: 'three',
 31: 'weeks',
 32: 'up',
 33: 'out',
 34: 'him',
 35: 'was',
 36: 'spent',
 37: 'learning',
 38: 'new',
 39: 'steps',
 40: 'long',
 41: 'away',
 42: 'left',
 43: 'friends',
 44: 'relations',
 45: 'when',
 46: 'wall',
 47: 'myself',
 48: 'nice',
 49: 'just',
 50: 'dancing',
 51: 'merry',
 52: 'tipped',
 53: 'me',
 54: 'soon',
 55: 'time',
 56: 'old',
 57: 'their',
 58: 'them',
 59: 'danced',
 60: 'dublin',
 61: 'an',
 62: 'put',
 63: 'leg',
 64: 'miss',
 65: 'fainted',
 66: 'from',
 67: 'town',
 68: 'athy',
 69: 'one',
 70: 'jeremy',
 71: 'lanigan',
 72: 'battered',
 73: 'hadnt',
 74: 'pound',
 75: 'father',
 76: 'died',
 77: 